# Hybrid Query: Vector + Graph RAG

Three query modes following [Vectors and Graphs: Better Together](https://neo4j.com/blog/developer/vectors-graphs-better-together/):

1. **Vector RAG** — pure semantic similarity on text chunks
2. **Graph RAG** — vector search + graph traversal via `VectorCypherRetriever`
3. **Hybrid** — merges context from both retrievers for a comprehensive answer

### Setup

In [ ]:
import neo4j
import os
from dotenv import load_dotenv

from neo4j_graphrag.llm import OpenAILLM
from neo4j_graphrag.embeddings.openai import OpenAIEmbeddings
from neo4j_graphrag.indexes import create_vector_index
from neo4j_graphrag.retrievers import VectorRetriever, VectorCypherRetriever
from neo4j_graphrag.generation import GraphRAG, RagTemplate

In [ ]:
load_dotenv()

NEO4J_URI = os.getenv('NEO4J_URI')
NEO4J_USERNAME = os.getenv('NEO4J_USERNAME')
NEO4J_PASSWORD = os.getenv('NEO4J_PASSWORD')

neo4j_driver = neo4j.GraphDatabase.driver(NEO4J_URI,
                auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

embedder = OpenAIEmbeddings()
llm = OpenAILLM(model_name="gpt-4o", model_params={"temperature": 0.0})

In [ ]:
# Create vector index (idempotent — safe to re-run)
create_vector_index(neo4j_driver, name="text_embeddings", label="Chunk",
                   embedding_property="embedding", dimensions=1536, similarity_fn="cosine")

In [ ]:
# Shared RAG prompt template
rag_template = RagTemplate(
    template='''Answer the Question using the following Context. Only respond with information mentioned in the Context. Do not inject any speculative information not mentioned.

# Question:
{query_text}

# Context:
{context}

# Examples:
{examples}

# Answer:
''',
    expected_inputs=['query_text', 'context', 'examples']
)

In [ ]:
# Query to test
q = "Placeholder."

---
### Option 1: Vector RAG

Pure semantic similarity search over `Chunk` text embeddings.

In [ ]:
vector_retriever = VectorRetriever(
    neo4j_driver,
    index_name="text_embeddings",
    embedder=embedder,
    return_properties=["text"],
)

vector_rag = GraphRAG(llm=llm, retriever=vector_retriever, prompt_template=rag_template)

vector_answer = vector_rag.search(q, retriever_config={'top_k': 5}).answer
print("=== Vector RAG ===")
print(vector_answer)

---
### Option 2: Graph RAG (VectorCypher)

Vector search as the entry point, then traverse 1-2 hops through entity relationships
to enrich the context with structured graph information.

In [ ]:
graph_retriever = VectorCypherRetriever(
    neo4j_driver,
    index_name="text_embeddings",
    embedder=embedder,
    retrieval_query="""
//1) Go out 1-2 hops in the entity graph and get relationships
WITH node AS chunk
MATCH (chunk)<-[:FROM_CHUNK]-(entity)-[relList:!FROM_CHUNK]-{1,2}(nb)
UNWIND relList AS rel

//2) collect relationships and text chunks
WITH collect(DISTINCT chunk) AS chunks, collect(DISTINCT rel) AS rels

//3) format and return context
RETURN apoc.text.join([c in chunks | c.text], '\n') +
  apoc.text.join([r in rels |
  startNode(r).name+' - '+type(r)+' '+r.details+' -> '+endNode(r).name],
  '\n') AS info
"""
)

graph_rag = GraphRAG(llm=llm, retriever=graph_retriever, prompt_template=rag_template)

graph_answer = graph_rag.search(q, retriever_config={'top_k': 5}).answer
print("=== Graph RAG ===")
print(graph_answer)

---
### Option 3: Hybrid Query

Combines **both** retrieval strategies:
- Vector retriever provides semantically relevant text chunks
- Graph retriever adds structured entity/relationship context

The merged context is sent to the LLM in a single prompt for a more comprehensive answer.

In [ ]:
def hybrid_query(query: str, top_k: int = 5) -> str:
    """Run both vector and graph retrievers, merge context, and generate a single answer."""

    # 1. Retrieve context from both sources
    vector_context = vector_retriever.search(query_text=query, top_k=top_k)
    graph_context = graph_retriever.search(query_text=query, top_k=top_k)

    # 2. Format vector context (text chunks)
    vector_texts = []
    for item in vector_context.items:
        text = item.content
        if text:
            vector_texts.append(text)
    vector_section = "\n---\n".join(vector_texts)

    # 3. Format graph context (entity relationships)
    graph_texts = []
    for item in graph_context.items:
        text = item.content
        if text:
            graph_texts.append(text)
    graph_section = "\n---\n".join(graph_texts)

    # 4. Build merged context
    merged_context = f"""== Text Chunks (Semantic Search) ==
{vector_section}

== Entity Relationships (Graph Traversal) ==
{graph_section}"""

    # 5. Build prompt and call LLM directly
    prompt = rag_template.format(query_text=query, context=merged_context, examples='')
    response = llm.invoke(prompt)
    return response.content


hybrid_answer = hybrid_query(q)
print("=== Hybrid Query ===")
print(hybrid_answer)

---
### Compare Results

In [ ]:
print("=" * 80)
print("VECTOR RAG")
print("=" * 80)
print(vector_answer)
print()
print("=" * 80)
print("GRAPH RAG")
print("=" * 80)
print(graph_answer)
print()
print("=" * 80)
print("HYBRID (VECTOR + GRAPH)")
print("=" * 80)
print(hybrid_answer)